# Annotation tool to create/correct 3D training labels

In [ ]:
import os
import numpy as np
from improc.io import parse_collection, DCAccessor
DCAccessor.register()
from holoviews import opts

from inter_view.dashboards import OrthoAnnotationDashBoard

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# config

In [ ]:
basedir = '../../data/3D'
data_pattern = '{subdir}/{fname}-T{time:04d}.{ext}'
index = ['subdir', 'time']

channel_config = {'nuclei':{'cmap':'gray'},
                  'nuclei_annot':{'cmap':'glasbey_hv_16bit', 'raster_aggregator':'first', 'intensity_bounds':(-2**15,2**15-1), 'slider_limits':(-2**15,2**15-1), 'bitdepth':'int16', 'opacity':0.5},}

def set_inactive_tool(plot, element):
    plot.state.toolbar.active_inspect = None

opts.defaults(opts.Image('channel.nuclei', frame_width=1000),
              opts.Image('channel.nuclei_annot', frame_width=1000, clipping_colors={'min': (0, 0, 0, 0)}, clim=(0,2**16-1), hooks=[set_inactive_tool], tools=['hover'], show_title=False),
             )

# parse files

In [ ]:
df = parse_collection(os.path.join(basedir, data_pattern), index)
df = df.dc[['nuclei', 'nuclei_annot']]

df

# interactive dashboard
To start drawing select the 'Freehand Drawing Tool' in the toolbar on the right of the image.

- drawing label:
    - -1: un-annotated (does not contribute to the training loss)
    - 0: background
    - [1...] instance labels. last one in the list is always unused
- on click
    - \-: does nothing
    - pick label (pipette): sets label at the clicked position as drawing label
    - fill label (pot filling): fill the label at the clicked label with the current drawing labels (discontinuous)
- lock
    - background: prevent from drawing over the existing background
    - foreground: prevents from drawing over the existing labels
- draw in 3D: draw with thickness in the 3rd dimension. best used to draw an initial "ball" at the center of each nuclei
- save: saves the current annotation. The current annotation is also automatically saved when loading a new image
- discard changes: Cancels all changes made since the last save (i.e. not a granual ctrl+z!)

In [ ]:
db = OrthoAnnotationDashBoard(df=df,
                           multi_select_levels=['subdir'],
                           channel_config=channel_config,
                           composite_channels=['nuclei'],
                           overlay_channels=['nuclei_annot'],
                           spacing=(2,0.26,0.26),
                           annot_channel='nuclei_annot')

db.panel().servable()